# Homework 1: Perceptrons
### Due Thursday, January 17, 2019, 5pm
#### Jesse Zhu
#### ML-W2019

Using the MNIST hand-written digits dataset, we aim to use perceptrons to categorize the labeled data through supervised learning. As this is a single-layer, the result is not expected to be perfect, but should be noticeably better than random. Each data point will be 785 values between 0 and 1, representing the grayscale value of each pixel in the original image. These values will be used to update our weight values over the course of 50 epochs, resulting in our 10 perceptrons (digits 0 to 9) being trained at the end. Finally, we will run test data through these trained perceptrons and take the maximum value of the 10 perceptrons as the predicted result.

In [40]:
#import pandas as pd
import numpy as np
#import random

In [30]:
#IMPORT DATA#################
testfile = "mnist_test.csv"
trainfile = "mnist_train.csv"

# test_data = pd.read_csv(testfile)
# train_data = pd.read_csv(trainfile)

#Data is 785 columns by N rows. First column = Label (0-9), others = 0:255
test_data = np.genfromtxt(testfile, skip_header=True, delimiter=',')
train_data = np.genfromtxt(trainfile, skip_header=True, delimiter=',')

In [46]:
#PREPROCESSING###############

#SCALE by 255
test_data[:,1:] /= 255
train_data[:,1:] /= 255
print((test_data[0,:]))

[2.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.45490196
 0.49019608 0.67058824 1.         1.         0.5

In [51]:
#SHUFFLE

#print((test_data[0,:]))
# random.uniform(0, 1)

#785,10

[[1.18598868 0.94382238 0.15674706 1.18448859 0.99081288]]


Appendix of Resources
1. Matrix multiplication
https://stackoverflow.com/questions/21562986/numpy-matrix-vector-multiplication
'''a = np.random.rand(1,3)
b = np.random.rand(3,5)
print(a@b)'''

